In [154]:
import pandas as pd
from pathlib import Path 

import dhlab as dh 

from norn.imag_corpus import make_imagination_corpus

# Last inn ImagiNation metadata

In [127]:
imagination_metadata = make_imagination_corpus()  # Får en dataramme med 24245 rader etter 41 sekunder


In [ ]:

imagination_metadata.drop_duplicates("urn")  # Gir en dataramme med 22946 rader, samme antall som i sprakbankdb2.lx.nb.no:/data/db/ft/project_metadata.db
# Unngå å droppe duplikater fordi vi mister dobbeltkategorier for oversatt litteratur

In [ ]:

imagination_metadata


# Last inn korpusfiler

In [240]:
# Kjør hele prosessen 

from hmac import new


corpusfiles = [
    "Data/1800-1839.xlsx",
    "Data/1840-1869.xlsx"
]

for cfile in corpusfiles:
    path = Path(cfile)
    corpus = pd.read_excel(path)
    corpus_with_metadata = corpus.merge(imagination_metadata, how="left", on="urn")
    missing_metadata = corpus[corpus_with_metadata.dhlabid.isna()]
    try: 
        assert missing_metadata.empty
    except AssertionError:
        print("OBS! Følgende dokumenter i NORN-korpuset mangler metadata fra ImagiNation: \n{}\n".format(missing_metadata))
        print("Henter metadata fra dhlab-APIet")
        c = dh.Corpus().from_identifiers(missing_metadata.urn)

        # Legg til kategori
        genres = c.frame.genres.unique()
        cats = []
        for g in genres: 
            cats.extend(corpus_with_metadata.category[corpus_with_metadata.genres == g].unique())
        mapping = dict(zip(genres, cats))
        print("Mapping av sjanger til ImagiNation-kategori: \n{}\n".format(mapping))
        c.frame["category"] = c.frame.genres.map(mapping)
        # Overskriv radene som mangler metadata fra ImagiNation
        corpus_with_metadata.loc[missing_metadata.index, corpus_with_metadata.columns] = c.frame[corpus_with_metadata.columns].values
    new_file = path.parent / (path.stem + "_metadata.xlsx")
    corpus_with_metadata.to_excel(new_file)
    print("Lagret data i ny fil {}".format(new_file))



OBS! Følgende dokumenter i NORN-korpuset mangler metadata fra ImagiNation: 
                                    urn
63  URN:NBN:no-nb_digibok_2015060948126
70  URN:NBN:no-nb_digibok_2009071010004
98  URN:NBN:no-nb_digibok_2007072703001

Henter metadata fra dhlab-APIet
Mapping av sjanger til ImagiNation-kategori: 
{'poetry': 'Diktning: Lyrikk'}

Lagret data i ny fil Data/1800-1839_metadata.xlsx
Lagret data i ny fil Data/1840-1869_metadata.xlsx


# KODEDUMP

Utkast og utprøving av kode. Ikke kjør på nytt


In [128]:
corpusfile_1800 = "Data/1800-1839.xlsx"
corpusfile_1840 = "Data/1840-1869.xlsx"

#consensus_corpus = "Konsensus_flat.xlsx"

In [144]:
corpusfile = corpusfile_1800

In [145]:
corpus = pd.read_excel(corpusfile)

In [146]:
corpus

,urn
0,URN:NBN:no-nb_digibok_2016101828011
1,URN:NBN:no-nb_digibok_2018071248006
2,URN:NBN:no-nb_digibok_2016101928009
3,URN:NBN:no-nb_digibok_2014102208131
4,URN:NBN:no-nb_digibok_2018042326017
...,...
96,URN:NBN:no-nb_digibok_2016100526001
97,URN:NBN:no-nb_digibok_2007102402004
98,URN:NBN:no-nb_digibok_2007072703001
99,URN:NBN:no-nb_digibok_2019102226008


In [201]:
corpus_with_metadata = corpus.merge(imagination_metadata, how="left", on="urn")

In [202]:
# Dokumenter i NORN-korpuset som ikker er i ImagiNation 
missing_metadata = corpus[corpus_with_metadata.dhlabid.isna()]

In [203]:
try: 
    assert missing_metadata.empty
except AssertionError: 
    print("Følgende dokumenter i NORN-korpuset mangler metadata fra ImagiNation: \n{}\n".format(missing_metadata))

Følgende dokumenter i NORN-korpuset mangler metadata fra ImagiNation: 
                                    urn
63  URN:NBN:no-nb_digibok_2015060948126
70  URN:NBN:no-nb_digibok_2009071010004
98  URN:NBN:no-nb_digibok_2007072703001



In [204]:
# Hent metadata fra dhlab-APIet for dokumenter som mangler metadata fra ImagiNation
c = dh.Corpus().from_identifiers(missing_metadata.urn)

# Legg til kategori
genres = c.frame.genres.unique()
cats = []
for g in genres: 
    cats.extend(corpus_with_metadata.category[corpus_with_metadata.genres == g].unique())

mapping = dict(zip(genres, cats))

print(mapping)
c.frame["category"] = c.frame.genres.map(mapping)
c

{'poetry': 'Diktning: Lyrikk'}


,urn,dhlabid,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype,ocr_creator,ocr_timestamp,category
0,URN:NBN:no-nb_digibok_2007072703001,100628186.0,En norsk Höst : Et Digt,"Zetlitz , Jens / Liunge , Poul Munkgaard / Pop...",oai:nb.bibsys.no:999912973004702202,30587a45784a22b45a099dbb9c38d98a,,,18000101.0,1800.0,"Trykt paa Boghandler P. M. Liunges Forlag, hos...",dan,Norsk litteratur / Dansk litteratur / 1700-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
1,URN:NBN:no-nb_digibok_2009071010004,100629422.0,Fjeldblomster,"Munch , Johan Storm / Lehmann , Jacob / Wedel-...",oai:nb.bibsys.no:990002811074702202,98e9799021673544030af837b374eb73,,,18130101.0,1813.0,Trykt hos Jacob Lehmann,dan,Norsk litteratur / 1800-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
2,URN:NBN:no-nb_digibok_2015060948126,100632718.0,Jonas Reins samlede Digte . 2 den Deel,"Rein , Jonas / Seidelin , Klaus Henrik",oai:nb.bibsys.no:999328669484702202,3fed386a693235bd0ba522186dbce9d0,,,18020101.0,1802.0,Forlagt af Arntzen & Hartier. Trykt hos K. H. ...,dan,Norsk litteratur / 1800-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk


In [231]:
# Funket ikke som ønsket 
#pd.merge(left = corpus_with_metadata, right=c.frame, on=list(corpus_with_metadata.columns), how="outer")

In [232]:
# Funket ikke som ønsket
#pd.concat([corpus_with_metadata, c.frame], axis=0, join="inner").dropna().reset_index(drop=True)

In [227]:
# Overskriv radene i korpuset 
corpus_with_metadata.loc[missing_metadata.index, corpus_with_metadata.columns] = c.frame[corpus_with_metadata.columns].values


In [230]:
corpus_with_metadata.iloc[missing_metadata.index]

,urn,dhlabid,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype,ocr_creator,ocr_timestamp,category
63,URN:NBN:no-nb_digibok_2007072703001,100628186.0,En norsk Höst : Et Digt,"Zetlitz , Jens / Liunge , Poul Munkgaard / Pop...",oai:nb.bibsys.no:999912973004702202,30587a45784a22b45a099dbb9c38d98a,,,18000101.0,1800.0,"Trykt paa Boghandler P. M. Liunges Forlag, hos...",dan,Norsk litteratur / Dansk litteratur / 1700-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
70,URN:NBN:no-nb_digibok_2009071010004,100629422.0,Fjeldblomster,"Munch , Johan Storm / Lehmann , Jacob / Wedel-...",oai:nb.bibsys.no:990002811074702202,98e9799021673544030af837b374eb73,,,18130101.0,1813.0,Trykt hos Jacob Lehmann,dan,Norsk litteratur / 1800-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
98,URN:NBN:no-nb_digibok_2015060948126,100632718.0,Jonas Reins samlede Digte . 2 den Deel,"Rein , Jonas / Seidelin , Klaus Henrik",oai:nb.bibsys.no:999328669484702202,3fed386a693235bd0ba522186dbce9d0,,,18020101.0,1802.0,Forlagt af Arntzen & Hartier. Trykt hos K. H. ...,dan,Norsk litteratur / 1800-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk


In [233]:
corpus_with_metadata

,urn,dhlabid,title,authors,oaiid,sesamid,isbn10,city,timestamp,year,publisher,langs,subjects,ddc,genres,literaryform,doctype,ocr_creator,ocr_timestamp,category
0,URN:NBN:no-nb_digibok_2016101828011,100633537.0,Tidsfordriv : Jule- og Nytaarsgave . Indeholde...,"/ Dahl , Chr . ( Christopher )",oai:nb.bibsys.no:999919857502202202,2c4a98789c11270ab307b045334d4ab0,,,18310101.0,1831.0,"Trykt paa J. Rasmussens Forlag, hos Chr. Dahl,...",dan,,,short story,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Dramatikk
1,URN:NBN:no-nb_digibok_2018071248006,100635381.0,"Fornuftpersen , eller Mundkurv paa Philosophie...","Allum , Hans",oai:nb.bibsys.no:999211643784702202,a1091a6d11597f2b618ead87a79b1c68,,,18320101.0,1832.0,"Trykt, paa J. C. Foslunds Forlag, i det Poppsk...",dan / nob,,,"humor, satire",Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
2,URN:NBN:no-nb_digibok_2016101928009,100633586.0,"Broder Ebben i Fødelandet , eller Amerikareise...","Allum , Hans / Abelsted , Jacob Christian",oai:nb.bibsys.no:990107838304702202,740962ffc2856a0878496f8a375f5158,,,18390101.0,1839.0,Trykt hos J.C. Abelsted,dan,Norsk litteratur / Amerikareisen / Norsk-ameri...,,drama,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Dramatikk
3,URN:NBN:no-nb_digibok_2014102208131,100632374.0,"Kongsberg , eller Norges Sølvværk : Et lyrisk ...","Allum , Hans / Fehr , G. L. ( Gottlieb Louis )...",oai:nb.bibsys.no:999424920134702202,07e649e30d33b3f29bed834febad97e3,,,18280101.0,1828.0,Trykt hos H. Gundersen,dan,Bergverksdrift,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
4,URN:NBN:no-nb_digibok_2018042326017,100635124.0,"Gjæringen , eller Alvor og Skjemt . Et Digt af...","Allum , Hans / Rode , Carl Ferdinand ( Carl Fe...",oai:nb.bibsys.no:999919918881902202,d6c077efd866b8d59f367720b2556ee3,,,18170101.0,1817.0,Trykt i Drammen hos Carl F. Rode,dan,Norsk litteratur / 1800-tallet,,"humor, satire",Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,URN:NBN:no-nb_digibok_2016100526001,100625446.0,Baldur : Nytaarsgave for Aaret 1829,"Winther , Hans Thøger / Hartmann , Søren Augus...",oai:nb.bibsys.no:990319109204702202,b618bd7bb17fac0cef3de7e166dda8f6,,,18280101.0,1828.0,"Forlagt af H.T. Winther, og trykt i hans Bogtr...",dan,,,,Skjønnlitteratur,digibok,dhlab,20221201.0,Diktning: Epikk
97,URN:NBN:no-nb_digibok_2007102402004,100628262.0,Samlede poetiske Forsøg,"Wolff , S. O. ( Simon Olaus ) / Winther , Hans...",oai:nb.bibsys.no:990306049154702202,f1733fdf0007d7b1611544405f8dcafe,,,18330101.0,1833.0,Paa H. T. Winthers Forlag. Trykt i Forlæggeren...,dan,Norsk litteratur / 1800-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
98,URN:NBN:no-nb_digibok_2015060948126,100632718.0,Jonas Reins samlede Digte . 2 den Deel,"Rein , Jonas / Seidelin , Klaus Henrik",oai:nb.bibsys.no:999328669484702202,3fed386a693235bd0ba522186dbce9d0,,,18020101.0,1802.0,Forlagt af Arntzen & Hartier. Trykt hos K. H. ...,dan,Norsk litteratur / 1800-tallet,,poetry,Skjønnlitteratur,digibok,dhlab,20221209.0,Diktning: Lyrikk
99,URN:NBN:no-nb_digibok_2019102226008,100627381.0,Norske Noveller : Første Bind : Præsten i Thel...,"Aalholm , N. M. ( Niels Matthias )",oai:nb.bibsys.no:999511172384702202,c5dc957556f7ccad1ac90568c5a4e06f,,,18380101.0,1838.0,"Guldberg & Dzwonkowski, Trykt i Forlæggernes O...",dan,Norsk litteratur / 1800-tallet,,short story,Skjønnlitteratur,digibok,dhlab,20221201.0,Diktning: Epikk


In [234]:
path = Path(corpusfile)

In [235]:
corpus_with_metadata.to_excel(path.parent / (path.stem + "_metadata.xlsx"))